In [1]:
from cobra.io import read_sbml_model
from six import string_types
import re

In [2]:
yali = read_sbml_model("../ModelFiles/xml/iYali.xml")

In [3]:
pattern = re.compile('s_[0-9]+')
unmapped_mets = [met for met in yali.metabolites if re.findall(pattern, met.id)]
len(unmapped_mets)

385

In [4]:
pattern = re.compile('y[0-9]+')
unmapped_rxns = [rxn for rxn in yali.reactions if re.findall(pattern, rxn.id)]
len(unmapped_rxns)

1543

In [5]:
iMM904 = read_sbml_model("../../../Downloads/iMM904.xml")

If the name of a metabolite in iYali matches a name in iMM904 then transfer the BiGG and MetaNetX IDs of the latter to the former.

In [6]:
duplicate_mapping = []
mapped_from_to = 'Based on its name {}, {} can be mapped to {}, {}'
for met_a in unmapped_mets:
    for met_b in iMM904.metabolites:
        if met_a.name.lower() == met_b.name.lower() and met_a.compartment == met_b.compartment:
            print(mapped_from_to.format(met_a.name, met_a.id, met_b.name, met_b.id))
            met_a.annotation.setdefault('bigg.metabolite', met_b.annotation['bigg.metabolite'])
            if 'metanetx.chemical' in met_b.annotation:
                met_a.annotation.setdefault('metanetx.chemical', met_b.annotation['metanetx.chemical'])
            met_a.annotation["standard.metabolite"] = met_a.id
            try: 
                met_a.id = met_a.annotation["bigg.metabolite"] + "_" + met_a.compartment
                yali.repair()
            except ValueError:
                duplicate_mapping.append(met_a)

Based on its name 2-methylcitrate, s_0174 can be mapped to 2-Methylcitrate, 2mcit_m
Based on its name 5,10-methylenetetrahydrofolate, s_0306 can be mapped to 5,10-Methylenetetrahydrofolate, mlthf_c
Based on its name 5,10-methylenetetrahydrofolate, s_0307 can be mapped to 5,10-Methylenetetrahydrofolate, mlthf_m
Based on its name D-fructose, s_0553 can be mapped to D-Fructose, fru_c
Based on its name D-fructose, s_0554 can be mapped to D-Fructose, fru_e
Based on its name D-fructose 1,6-bisphosphate, s_0555 can be mapped to D-Fructose 1,6-bisphosphate, fdp_c
Based on its name D-fructose 1-phosphate, s_0556 can be mapped to D-Fructose 1-phosphate, f1p_c
Based on its name D-fructose 6-phosphate, s_0557 can be mapped to D-Fructose 6-phosphate, f6p_c
Based on its name D-mannose 1-phosphate, s_0573 can be mapped to D-Mannose 1-phosphate, man1p_c
Based on its name D-mannose 6-phosphate, s_0574 can be mapped to D-Mannose 6-phosphate, man6p_c
Based on its name L-homocysteine, s_1012 can be mapped

In [7]:
pattern = re.compile('s_[0-9]+')
unmapped_mets = [met for met in yali.metabolites if re.findall(pattern, met.id)]
len(unmapped_mets)

365

Since this was only semi-successful I'm going to try this again with fuzzy string search using the packages `Fuzzywuzzy` and `python-Levenshtein`.

In [8]:
from fuzzywuzzy import process

In [9]:
def transfer_IDs(met_a, met_b):
    mapped_from_to = 'Based on its name {}, {} can be mapped to {}, {} with the new ID: {}'
    met_a.annotation.setdefault('bigg.metabolite', met_b.annotation['bigg.metabolite'])
    if 'metanetx.chemical' in met_b.annotation:
        met_a.annotation.setdefault('metanetx.chemical', met_b.annotation['metanetx.chemical'])
    met_a.annotation["standard.metabolite"] = met_a.id
    try: 
        old_id = met_a.id
        met_a.id = met_a.annotation["bigg.metabolite"] + "_" + met_a.compartment
        yali.repair()
        print(mapped_from_to.format(met_a.name, old_id, met_b.name, met_b.id, met_a.id))
        print('.,-~^~-,'*10)
    except ValueError:
        print('Could not map metabolites because the new ID most likely already exists in the model.')

In [10]:
suggested_mappings = []
# Generate a dict of met.id:met.name for iMM904:
choice = {met: met.name for met in iMM904.metabolites}

for met_a in unmapped_mets:
    relevant_matches = []
    gen = process.extractWithoutOrder(met_a.name, choice)
    for tup in list(gen):        
        if tup[1] > 50 and tup[2].formula == met_a.formula:
            relevant_matches.append(tup)
    if not relevant_matches:
        continue
    suggested_mappings.append((met_a.id, met_a.name, met_a.compartment, relevant_matches))

In [11]:
suggested_mappings

[('s_0001',
  '(1-3)-beta-D-glucan',
  'ce',
  [('1 3 beta D Glucan C6H10O5', 95, <Metabolite 13BDglcn_c at 0x116f480f0>),
   ('1 3 beta D Glucan C6H10O5', 95, <Metabolite 13BDglcn_e at 0x116f3df60>),
   ('1 6 beta D Glucan C6H10O5', 89, <Metabolite 16BDglcn_c at 0x116f3d978>)]),
 ('s_0003',
  '(1-3)-beta-D-glucan',
  'e',
  [('1 3 beta D Glucan C6H10O5', 95, <Metabolite 13BDglcn_c at 0x116f480f0>),
   ('1 3 beta D Glucan C6H10O5', 95, <Metabolite 13BDglcn_e at 0x116f3df60>),
   ('1 6 beta D Glucan C6H10O5', 89, <Metabolite 16BDglcn_c at 0x116f3d978>)]),
 ('s_0004',
  '(1-6)-beta-D-glucan',
  'ce',
  [('1 3 beta D Glucan C6H10O5', 89, <Metabolite 13BDglcn_c at 0x116f480f0>),
   ('1 3 beta D Glucan C6H10O5', 89, <Metabolite 13BDglcn_e at 0x116f3df60>),
   ('1 6 beta D Glucan C6H10O5', 95, <Metabolite 16BDglcn_c at 0x116f3d978>)]),
 ('s_0012',
  '(2S,3R)-3-hydroxybutane-1,2,3-tricarboxylic acid',
  'm',
  [('2-Methylcitrate', 86, <Metabolite 2mcit_m at 0x116ebc550>),
   ('2 Hydroxybutane

In [12]:
a = yali.metabolites.get_by_id("s_0001")
b = iMM904.metabolites.get_by_id("13BDglcn_c")
transfer_IDs(a, b)

Based on its name (1-3)-beta-D-glucan, s_0001 can be mapped to 1 3 beta D Glucan C6H10O5, 13BDglcn_c with the new ID: 13BDglcn_ce
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [13]:
a = yali.metabolites.get_by_id("s_0003")
b = iMM904.metabolites.get_by_id("13BDglcn_e")
transfer_IDs(a, b)

Based on its name (1-3)-beta-D-glucan, s_0003 can be mapped to 1 3 beta D Glucan C6H10O5, 13BDglcn_e with the new ID: 13BDglcn_e
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [14]:
a = yali.metabolites.get_by_id("s_0004")
b = iMM904.metabolites.get_by_id("16BDglcn_c")
transfer_IDs(a, b)

Based on its name (1-6)-beta-D-glucan, s_0004 can be mapped to 1 6 beta D Glucan C6H10O5, 16BDglcn_c with the new ID: 16BDglcn_ce
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [15]:
a = yali.metabolites.get_by_id("s_0015")
b = iMM904.metabolites.get_by_id("argsuc_c")
transfer_IDs(a, b)

Based on its name (N(omega)-L-arginino)succinic acid, s_0015 can be mapped to N(omega)-(L-Arginino)succinate, argsuc_c with the new ID: argsuc_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [16]:
a = yali.metabolites.get_by_id("s_0086")
b = iMM904.metabolites.get_by_id("3ig3p_c")
transfer_IDs(a, b)

Based on its name 1-C-(indol-3-yl)glycerol 3-phosphate, s_0086 can be mapped to C'-(3-Indolyl)-glycerol 3-phosphate, 3ig3p_c with the new ID: 3ig3p_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [17]:
a = yali.metabolites.get_by_id("s_0116")
b = iMM904.metabolites.get_by_id("1p3h5c_c")
transfer_IDs(a, b)

Based on its name 1-pyrroline-3-hydroxy-5-carboxylic acid, s_0116 can be mapped to L 1 Pyrroline 3 hydroxy 5 carboxylate C5H6NO3, 1p3h5c_c with the new ID: 1p3h5c_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [18]:
a = yali.metabolites.get_by_id("s_0117")
b = iMM904.metabolites.get_by_id("1p3h5c_m")
transfer_IDs(a, b)

Based on its name 1-pyrroline-3-hydroxy-5-carboxylic acid, s_0117 can be mapped to L 1 Pyrroline 3 hydroxy 5 carboxylate C5H6NO3, 1p3h5c_m with the new ID: 1p3h5c_m
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [19]:
a = yali.metabolites.get_by_id("s_0123")
b = iMM904.metabolites.get_by_id("mi1345p_n")
transfer_IDs(a, b)

Based on its name 1D-myo-inositol 1,3,4,5-tetrakisphosphate, s_0123 can be mapped to 1D myo Inositol 1 3 4 5 tetrakisphosphate C6H8O18P4, mi1345p_n with the new ID: mi1345p_n
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [20]:
a = yali.metabolites.get_by_id("s_0124")
b = iMM904.metabolites.get_by_id("mi1456p_n")
transfer_IDs(a, b)

Based on its name 1D-myo-inositol 1,4,5,6-tetrakisphosphate, s_0124 can be mapped to 1D myo Inositol 1 4 5 6 tetrakisphosphate C6H8O18P4, mi1456p_n with the new ID: mi1456p_n
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [21]:
a = yali.metabolites.get_by_id("s_0125")
b = iMM904.metabolites.get_by_id("mi145p_n")
transfer_IDs(a, b)

Based on its name 1D-myo-inositol 1,4,5-trisphosphate, s_0125 can be mapped to 1D myo Inositol 1 4 5 trisphosphate C6H9O15P3, mi145p_n with the new ID: mi145p_n
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [22]:
a = yali.metabolites.get_by_id("s_0146")
b = iMM904.metabolites.get_by_id("alac__S_m")
transfer_IDs(a, b)

Based on its name 2-acetyllactic acid, s_0146 can be mapped to (S)-2-Acetolactate, alac__S_m with the new ID: alac__S_m
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [23]:
a = yali.metabolites.get_by_id("s_0154")
b = iMM904.metabolites.get_by_id("2hpmhmbq_m")
transfer_IDs(a, b)

Based on its name 2-hexaprenyl-5-hydroxy-6-methoxy-3-methyl-1,4-benzoquinone, s_0154 can be mapped to 2 hexaprenyl 3 methyl 5 hydroxy 6 methoxy 1 4 benzoquinone C38H56O4, 2hpmhmbq_m with the new ID: 2hpmhmbq_m
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [24]:
a = yali.metabolites.get_by_id("s_0155")
b = iMM904.metabolites.get_by_id("2hp6mbq_m")
transfer_IDs(a, b)

Based on its name 2-hexaprenyl-6-methoxy-1,4-benzoquinone, s_0155 can be mapped to 2 Hexaprenyl 6 methoxy 1 4 benzoquinone C37H54O3, 2hp6mbq_m with the new ID: 2hp6mbq_m
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [25]:
a = yali.metabolites.get_by_id("s_0156")
b = iMM904.metabolites.get_by_id("2hpmmbq_m")
transfer_IDs(a, b)

Based on its name 2-hexaprenyl-6-methoxy-3-methyl-1,4-benzoquinone, s_0156 can be mapped to 2 hexaprenyl 3 methyl 6 methoxy 1 4 benzoquinone C38H56O3, 2hpmmbq_m with the new ID: 2hpmmbq_m
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [26]:
a = yali.metabolites.get_by_id("s_0193")
b = iMM904.metabolites.get_by_id("35ccmp_c")
transfer_IDs(a, b)

Based on its name 3',5'-cyclic CMP, s_0193 can be mapped to 3  5  Cyclic CMP C9H11N3O7P, 35ccmp_c with the new ID: 35ccmp_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [27]:
a = yali.metabolites.get_by_id("s_0234")
b = iMM904.metabolites.get_by_id("3mbald_c")
transfer_IDs(a, b)

Based on its name 3-methylbutanal, s_0234 can be mapped to 3 Methylbutanal C5H10O, 3mbald_c with the new ID: 3mbald_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [28]:
a = yali.metabolites.get_by_id("s_0236")
b = iMM904.metabolites.get_by_id("3mbald_m")
transfer_IDs(a, b)

Based on its name 3-methylbutanal, s_0236 can be mapped to 3 Methylbutanal C5H10O, 3mbald_m with the new ID: 3mbald_m
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [29]:
a = yali.metabolites.get_by_id("s_0262")
b = iMM904.metabolites.get_by_id("44mctr_c")
transfer_IDs(a, b)

Based on its name 4,4-dimethyl-5alpha-cholesta-8,14,24-trien-3beta-ol, s_0262 can be mapped to 4 4 dimethylcholesta 8 14 24 trienol C29H46O, 44mctr_c with the new ID: 44mctr_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [30]:
a = yali.metabolites.get_by_id("s_0296")
b = iMM904.metabolites.get_by_id("4mzym_c")
transfer_IDs(a, b)

Based on its name 4alpha-methylzymosterol, s_0296 can be mapped to 4 methylzymosterol C28H46O, 4mzym_c with the new ID: 4mzym_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [31]:
a = yali.metabolites.get_by_id("s_0346")
b = iMM904.metabolites.get_by_id("ahdt_c")
transfer_IDs(a, b)

Based on its name 7,8-dihydroneopterin 3'-triphosphate, s_0346 can be mapped to 2-Amino-4-hydroxy-6-(erythro-1,2,3-trihydroxypropyl)dihydropteridine triphosphate, ahdt_c with the new ID: ahdt_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [32]:
a = yali.metabolites.get_by_id("s_0389")
b = iMM904.metabolites.get_by_id("amp2p_c")
transfer_IDs(a, b)

Based on its name adenosine 2'-phosphate, s_0389 can be mapped to Adenosine 2  phosphate C10H14N5O7P, amp2p_c with the new ID: amp2p_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [33]:
a = yali.metabolites.get_by_id("s_0432")
b = iMM904.metabolites.get_by_id("asptrna_c")
transfer_IDs(a, b)

Based on its name Asp-tRNA(Asp), s_0432 can be mapped to L-Aspartyl-tRNA(Asp), asptrna_c with the new ID: asptrna_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [34]:
a = yali.metabolites.get_by_id("s_0433")
b = iMM904.metabolites.get_by_id("asptrna_m")
transfer_IDs(a, b)

Based on its name Asp-tRNA(Asp), s_0433 can be mapped to L-Aspartyl-tRNA(Asp), asptrna_m with the new ID: asptrna_m
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [35]:
a = yali.metabolites.get_by_id("s_0453")
b = iMM904.metabolites.get_by_id("btamp_c")
transfer_IDs(a, b)

Based on its name biotinyl-5'-AMP, s_0453 can be mapped to Biotinyl 5  AMP C20H27N7O9PS, btamp_c with the new ID: btamp_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [36]:
a = yali.metabolites.get_by_id("s_0548")
b = iMM904.metabolites.get_by_id("arab__D_c")
transfer_IDs(a, b)

Based on its name D-arabinose, s_0548 can be mapped to D Arabinose C5H10O5, arab__D_c with the new ID: arab__D_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [37]:
a = yali.metabolites.get_by_id("s_0600")
b = iMM904.metabolites.get_by_id("dca_c")
transfer_IDs(a, b)

Based on its name decanoate, s_0600 can be mapped to Decanoate (n-C10:0), dca_c with the new ID: dca_p
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [38]:
a = yali.metabolites.get_by_id("s_0664")
b = iMM904.metabolites.get_by_id("ergtrol_c")
transfer_IDs(a, b)

Based on its name ergosta-5,7,24(28)-trien-3beta-ol, s_0664 can be mapped to Ergosta 5 7 24 28  trienol C28H44O, ergtrol_c with the new ID: ergtrol_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [39]:
a = yali.metabolites.get_by_id("s_0670")
b = iMM904.metabolites.get_by_id("ergst3glc_c")
transfer_IDs(a, b)

Based on its name ergosterol 3-beta-D-glucoside, s_0670 can be mapped to Ergosterol 3 beta D glucoside C34H54O6, ergst3glc_c with the new ID: ergst3glc_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [40]:
a = yali.metabolites.get_by_id("s_0836")
b = iMM904.metabolites.get_by_id("hicit_m")
transfer_IDs(a, b)

Based on its name homoisocitrate, s_0836 can be mapped to Homoisocitrate C7H7O7, hicit_m with the new ID: hicit_m
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [41]:
a = yali.metabolites.get_by_id("s_0951")
b = iMM904.metabolites.get_by_id("phpyr_c")
transfer_IDs(a, b)

Based on its name keto-phenylpyruvate, s_0951 can be mapped to Phenylpyruvate, phpyr_c with the new ID: phpyr_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [42]:
a = yali.metabolites.get_by_id("s_1043")
b = iMM904.metabolites.get_by_id("srb__L_c")
transfer_IDs(a, b)

Based on its name L-sorbose, s_1043 can be mapped to L Sorbose C6H12O6, srb__L_c with the new ID: srb__L_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [43]:
a = yali.metabolites.get_by_id("s_1070")
b = iMM904.metabolites.get_by_id("ddca_c")
transfer_IDs(a, b)

Based on its name laurate, s_1070 can be mapped to Dodecanoate (n-C12:0), ddca_c with the new ID: ddca_p
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [44]:
a = yali.metabolites.get_by_id("s_1148")
b = iMM904.metabolites.get_by_id("mettrna_c")
transfer_IDs(a, b)

Based on its name Met-tRNA(Met), s_1148 can be mapped to L-Methionyl-tRNA (Met), mettrna_c with the new ID: mettrna_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [45]:
a = yali.metabolites.get_by_id("s_1149")
b = iMM904.metabolites.get_by_id("mettrna_m")
transfer_IDs(a, b)

Based on its name Met-tRNA(Met), s_1149 can be mapped to L-Methionyl-tRNA (Met), mettrna_m with the new ID: mettrna_m
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [46]:
a = yali.metabolites.get_by_id("s_1156")
b = iMM904.metabolites.get_by_id("mi13456p_c")
transfer_IDs(a, b)

Based on its name myo-inositol 1,3,4,5,6-pentakisphosphate, s_1156 can be mapped to 1D myo Inositol 1 3 4 5 6 pentakisphosphate C6H7O21P5, mi13456p_c with the new ID: mi13456p_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [47]:
a = yali.metabolites.get_by_id("s_1157")
b = iMM904.metabolites.get_by_id("mi13456p_n")
transfer_IDs(a, b)

Based on its name myo-inositol 1,3,4,5,6-pentakisphosphate, s_1157 can be mapped to 1D myo Inositol 1 3 4 5 6 pentakisphosphate C6H7O21P5, mi13456p_n with the new ID: mi13456p_n
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [48]:
a = yali.metabolites.get_by_id("s_1428")
b = iMM904.metabolites.get_by_id("sertrna_c")
transfer_IDs(a, b)

Based on its name Ser-tRNA(Ser), s_1428 can be mapped to L-Seryl-tRNA(Ser), sertrna_c with the new ID: sertrna_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [49]:
a = yali.metabolites.get_by_id("s_1489")
b = iMM904.metabolites.get_by_id("thm_c")
transfer_IDs(a, b)

Based on its name thiamine, s_1489 can be mapped to Thiamin, thm_c with the new ID: thm_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [50]:
a = yali.metabolites.get_by_id("s_1541")
b = iMM904.metabolites.get_by_id("udpgal_c")
transfer_IDs(a, b)

Based on its name UDP-D-galactose, s_1541 can be mapped to UDPgalactose, udpgal_c with the new ID: udpgal_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [51]:
a = yali.metabolites.get_by_id("s_1542")
b = iMM904.metabolites.get_by_id("udpgal_g")
transfer_IDs(a, b)

Based on its name UDP-D-galactose, s_1542 can be mapped to UDPgalactose, udpgal_g with the new ID: udpgal_g
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [52]:
a = yali.metabolites.get_by_id("s_1565")
b = iMM904.metabolites.get_by_id("xmp_c")
transfer_IDs(a, b)

Based on its name xanthosine-5-phosphate, s_1565 can be mapped to Xanthosine 5'-phosphate, xmp_c with the new ID: xmp_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [53]:
a = yali.metabolites.get_by_id("s_2819")
b = iMM904.metabolites.get_by_id("hdcoa_c")
transfer_IDs(a, b)

Based on its name palmitoleoyl-CoA, s_2819 can be mapped to Hexadecenoyl-CoA (n-C16:1CoA), hdcoa_c with the new ID: hdcoa_erm
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [54]:
a = yali.metabolites.get_by_id("s_2832")
b = iMM904.metabolites.get_by_id("ddca_c")
transfer_IDs(a, b)

Based on its name laurate, s_2832 can be mapped to Dodecanoate (n-C12:0), ddca_c with the new ID: ddca_erm
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [55]:
a = yali.metabolites.get_by_id("s_2841")
b = iMM904.metabolites.get_by_id("ddca_c")
transfer_IDs(a, b)

Based on its name laurate, s_2841 can be mapped to Dodecanoate (n-C12:0), ddca_c with the new ID: ddca_lp
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [56]:
a = yali.metabolites.get_by_id("s_2849")
b = iMM904.metabolites.get_by_id("hdcoa_c")
transfer_IDs(a, b)

Based on its name palmitoleoyl-CoA, s_2849 can be mapped to Hexadecenoyl-CoA (n-C16:1CoA), hdcoa_c with the new ID: hdcoa_lp
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [57]:
a = yali.metabolites.get_by_id("s_2854")
b = iMM904.metabolites.get_by_id("hdcoa_c")
transfer_IDs(a, b)

Based on its name palmitoleoyl-CoA, s_2854 can be mapped to Hexadecenoyl-CoA (n-C16:1CoA), hdcoa_c with the new ID: hdcoa_p
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [58]:
a = yali.metabolites.get_by_id("s_2877")
b = iMM904.metabolites.get_by_id("hdcoa_c")
transfer_IDs(a, b)

Based on its name palmitoleoyl-CoA, s_2877 can be mapped to Hexadecenoyl-CoA (n-C16:1CoA), hdcoa_c with the new ID: hdcoa_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [59]:
a = yali.metabolites.get_by_id("s_3498")
b = iMM904.metabolites.get_by_id("mi145p_c")
transfer_IDs(a, b)

Based on its name 1D-myo-inositol 1,4,5-trisphosphate, s_3498 can be mapped to 1D myo Inositol 1 4 5 trisphosphate C6H9O15P3, mi145p_c with the new ID: mi145p_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [60]:
a = yali.metabolites.get_by_id("s_0235")
b = iMM904.metabolites.get_by_id("3mbald_e")
transfer_IDs(a, b)

Based on its name 3-methylbutanal, s_0235 can be mapped to 3 Methylbutanal C5H10O, 3mbald_e with the new ID: 3mbald_e
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [61]:
a = yali.metabolites.get_by_id("s_0549")
b = iMM904.metabolites.get_by_id("arab__D_e")
transfer_IDs(a, b)

Based on its name D-arabinose, s_0549 can be mapped to D Arabinose C5H10O5, arab__D_e with the new ID: arab__D_e
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [62]:
a = yali.metabolites.get_by_id("s_0597")
b = iMM904.metabolites.get_by_id("dca_e")
transfer_IDs(a, b)

Based on its name decanoate, s_0597 can be mapped to Decanoate (n-C10:0), dca_e with the new ID: dca_e
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [63]:
a = yali.metabolites.get_by_id("s_1044")
b = iMM904.metabolites.get_by_id("srb__L_e")
transfer_IDs(a, b)

Based on its name L-sorbose, s_1044 can be mapped to L Sorbose C6H12O6, srb__L_e with the new ID: srb__L_e
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [64]:
a = yali.metabolites.get_by_id("s_1067")
b = iMM904.metabolites.get_by_id("ddca_e")
transfer_IDs(a, b)

Based on its name laurate, s_1067 can be mapped to Dodecanoate (n-C12:0), ddca_e with the new ID: ddca_e
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [65]:
a = yali.metabolites.get_by_id("s_1490")
b = iMM904.metabolites.get_by_id("thm_e")
transfer_IDs(a, b)

Based on its name thiamine, s_1490 can be mapped to Thiamin, thm_e with the new ID: thm_e
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [66]:
a = yali.metabolites.get_by_id("s_0002")
b = iMM904.metabolites.get_by_id("13BDglcn_c")
transfer_IDs(a, b)

Based on its name (1-3)-beta-D-glucan, s_0002 can be mapped to 1 3 beta D Glucan C6H10O5, 13BDglcn_c with the new ID: 13BDglcn_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [67]:
a = yali.metabolites.get_by_id("s_0595")
b = iMM904.metabolites.get_by_id("dca_c")
transfer_IDs(a, b)

Based on its name decanoate, s_0595 can be mapped to Decanoate (n-C10:0), dca_c with the new ID: dca_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [68]:
a = yali.metabolites.get_by_id("s_1065")
b = iMM904.metabolites.get_by_id("ddca_c")
transfer_IDs(a, b)

Based on its name laurate, s_1065 can be mapped to Dodecanoate (n-C12:0), ddca_c with the new ID: ddca_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [69]:
a = yali.metabolites.get_by_id("s_3320")
b = iMM904.metabolites.get_by_id("hdcoa_c")
transfer_IDs(a, b)

Based on its name palmitoleoyl-CoA, s_3320 can be mapped to Hexadecenoyl-CoA (n-C16:1CoA), hdcoa_c with the new ID: hdcoa_mm
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [70]:
pattern = re.compile('s_[0-9]+')
unmapped_mets = [met for met in yali.metabolites if re.findall(pattern, met.id)]
len(unmapped_mets)

307

In [71]:
from fuzzywuzzy import fuzz

In [72]:
suggested_mappings_2 = []
# Generate a dict of met.id:met.name for iMM904:
choice = {met: met.name for met in iMM904.metabolites}

for met_a in unmapped_mets:
    relevant_matches = []
    for key in choice:
        score = fuzz.token_set_ratio(met_a.name, choice[key])
        if score > 90:
            tup = (score, choice[key], key.id)
            relevant_matches.append(tup)
    if not relevant_matches:
        continue
    suggested_mappings_2.append((met_a.id, met_a.name, met_a.compartment, relevant_matches))

In [73]:
a = yali.metabolites.get_by_id("s_0194")
b = iMM904.metabolites.get_by_id("35cdamp_c")
transfer_IDs(a, b)

Based on its name 3',5'-cyclic dAMP, s_0194 can be mapped to 3  5  Cyclic dAMP C10H11N5O5P, 35cdamp_c with the new ID: 35cdamp_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [74]:
a = yali.metabolites.get_by_id("s_0196")
b = iMM904.metabolites.get_by_id("35cimp_c")
transfer_IDs(a, b)

Based on its name 3',5'-cyclic IMP, s_0196 can be mapped to 3  5  Cyclic IMP C10H10N4O7P, 35cimp_c with the new ID: 35cimp_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [75]:
a = yali.metabolites.get_by_id("s_0412")
b = iMM904.metabolites.get_by_id("gam6p_c")
transfer_IDs(a, b)

Based on its name alpha-D-glucosamine 6-phosphate, s_0412 can be mapped to D-Glucosamine 6-phosphate, gam6p_c with the new ID: gam6p_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [76]:
a = yali.metabolites.get_by_id("s_0443")
b = iMM904.metabolites.get_by_id("macchitppdol_g")
transfer_IDs(a, b)

Based on its name beta-D-mannosyldiacetylchitobiosyldiphosphodolichol, s_0443 can be mapped to Beta D Mannosyldiacetylchitobiosyldiphosphodolichol C37H64N2O22P2, macchitppdol_g with the new ID: macchitppdol_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [77]:
a = yali.metabolites.get_by_id("s_0444")
b = iMM904.metabolites.get_by_id("macchitppdol_g")
transfer_IDs(a, b)

Based on its name beta-D-mannosyldiacetylchitobiosyldiphosphodolichol, s_0444 can be mapped to Beta D Mannosyldiacetylchitobiosyldiphosphodolichol C37H64N2O22P2, macchitppdol_g with the new ID: macchitppdol_g
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [78]:
a = yali.metabolites.get_by_id("s_0816")
b = iMM904.metabolites.get_by_id("hexccoa_c")
transfer_IDs(a, b)

Based on its name hexacosanoyl-CoA, s_0816 can be mapped to Hexacosanoyl CoA  n C260CoA  C47H82N7O17P3S, hexccoa_c with the new ID: hexccoa_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [79]:
a = yali.metabolites.get_by_id("s_0817")
b = iMM904.metabolites.get_by_id("hexccoa_c")
transfer_IDs(a, b)

Based on its name hexacosanoyl-CoA, s_0817 can be mapped to Hexacosanoyl CoA  n C260CoA  C47H82N7O17P3S, hexccoa_c with the new ID: hexccoa_er
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [80]:
a = yali.metabolites.get_by_id("s_0819")
b = iMM904.metabolites.get_by_id("hexccoa_c")
transfer_IDs(a, b)

Based on its name hexacosanoyl-CoA, s_0819 can be mapped to Hexacosanoyl CoA  n C260CoA  C47H82N7O17P3S, hexccoa_c with the new ID: hexccoa_p
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [81]:
a = yali.metabolites.get_by_id("s_1189")
b = iMM904.metabolites.get_by_id("acgam1p_c")
transfer_IDs(a, b)

Based on its name N-acetyl-alpha-D-glucosamine 1-phosphate, s_1189 can be mapped to N-Acetyl-D-glucosamine 1-phosphate, acgam1p_c with the new ID: acgam1p_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [82]:
a = yali.metabolites.get_by_id("s_1254")
b = iMM904.metabolites.get_by_id("ocACP_m")
transfer_IDs(a, b)

Based on its name octanoyl-ACP, s_1254 can be mapped to Octanoyl-ACP (n-C8:0ACP), ocACP_m with the new ID: ocACP_m
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [83]:
a = yali.metabolites.get_by_id("s_1479")
b = iMM904.metabolites.get_by_id("ttccoa_c")
transfer_IDs(a, b)

Based on its name tetracosanoyl-CoA, s_1479 can be mapped to Tetracosanoyl CoA   n C240CoA  C45H78N7O17P3S, ttccoa_c with the new ID: ttccoa_c
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [84]:
a = yali.metabolites.get_by_id("s_1480")
b = iMM904.metabolites.get_by_id("ttccoa_c")
transfer_IDs(a, b)

Based on its name tetracosanoyl-CoA, s_1480 can be mapped to Tetracosanoyl CoA   n C240CoA  C45H78N7O17P3S, ttccoa_c with the new ID: ttccoa_er
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [85]:
a = yali.metabolites.get_by_id("s_1482")
b = iMM904.metabolites.get_by_id("ttccoa_c")
transfer_IDs(a, b)

Based on its name tetracosanoyl-CoA, s_1482 can be mapped to Tetracosanoyl CoA   n C240CoA  C45H78N7O17P3S, ttccoa_c with the new ID: ttccoa_p
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [86]:
a = yali.metabolites.get_by_id("s_2797")
b = iMM904.metabolites.get_by_id("ttccoa_c")
transfer_IDs(a, b)

Based on its name tetracosanoyl-CoA, s_2797 can be mapped to Tetracosanoyl CoA   n C240CoA  C45H78N7O17P3S, ttccoa_c with the new ID: ttccoa_erm
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [87]:
a = yali.metabolites.get_by_id("s_2810")
b = iMM904.metabolites.get_by_id("hdd2coa_x")
transfer_IDs(a, b)

Based on its name trans-hexadec-2-enoyl-CoA, s_2810 can be mapped to Trans-Hexadec-2-enoyl-CoA, hdd2coa_x with the new ID: hdd2coa_erm
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [88]:
a = yali.metabolites.get_by_id("s_2816")
b = iMM904.metabolites.get_by_id("hexccoa_c")
transfer_IDs(a, b)

Based on its name hexacosanoyl-CoA, s_2816 can be mapped to Hexacosanoyl CoA  n C260CoA  C47H82N7O17P3S, hexccoa_c with the new ID: hexccoa_erm
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [89]:
a = yali.metabolites.get_by_id("s_2865")
b = iMM904.metabolites.get_by_id("ttccoa_c")
transfer_IDs(a, b)

Based on its name tetracosanoyl-CoA, s_2865 can be mapped to Tetracosanoyl CoA   n C240CoA  C45H78N7O17P3S, ttccoa_c with the new ID: ttccoa_ce
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [90]:
a = yali.metabolites.get_by_id("s_2867")
b = iMM904.metabolites.get_by_id("hexccoa_c")
transfer_IDs(a, b)

Based on its name hexacosanoyl-CoA, s_2867 can be mapped to Hexacosanoyl CoA  n C260CoA  C47H82N7O17P3S, hexccoa_c with the new ID: hexccoa_ce
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [91]:
a = yali.metabolites.get_by_id("s_2874")
b = iMM904.metabolites.get_by_id("ttccoa_c")
transfer_IDs(a, b)

Based on its name tetracosanoyl-CoA, s_2874 can be mapped to Tetracosanoyl CoA   n C240CoA  C45H78N7O17P3S, ttccoa_c with the new ID: ttccoa_lp
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [92]:
a = yali.metabolites.get_by_id("s_2876")
b = iMM904.metabolites.get_by_id("hexccoa_c")
transfer_IDs(a, b)

Based on its name hexacosanoyl-CoA, s_2876 can be mapped to Hexacosanoyl CoA  n C260CoA  C47H82N7O17P3S, hexccoa_c with the new ID: hexccoa_lp
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [93]:
a = yali.metabolites.get_by_id("s_0413")
b = iMM904.metabolites.get_by_id("gam6p_c")
transfer_IDs(a, b)

Based on its name alpha-D-glucosamine 6-phosphate, s_0413 can be mapped to D-Glucosamine 6-phosphate, gam6p_c with the new ID: gam6p_e
.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,.,-~^~-,


In [94]:
pattern = re.compile('s_[0-9]+')
unmapped_mets = [met for met in yali.metabolites if re.findall(pattern, met.id)]
len(unmapped_mets)

286

In [95]:
pattern = re.compile('y[0-9]+')
unmapped_rxns = [rxn for rxn in yali.reactions if re.findall(pattern, rxn.id)]
len(unmapped_rxns)

1543

In [96]:
suggested_rxn_mappings = []

for rxn in yali.reactions:
    relevant_matches = []
    a_reactants = set([met.id for met in rxn.reactants])
    a_products = set([met.id for met in rxn.products])
    for rxn_cand in iMM904.reactions:
        b_reactants = set([met.id for met in rxn_cand.reactants])
        b_products = set([met.id for met in rxn_cand.products])
        if a_reactants != b_reactants:
            continue
        if a_products != b_products:
            continue
        tup = (rxn_cand.id, rxn_cand.name)
        relevant_matches.append(tup)
    if not relevant_matches:
        continue
    suggested_rxn_mappings.append((rxn.id, rxn.name, relevant_matches))

In [97]:
suggested_rxn_mappings

[('PRICIh',
  '1-(5-phosphoribosyl)-5-[(5-phosphoribosylamino)methylideneamino)imidazole-4-carboxamide isomerase',
  [('PRMICI',
    '1-(5-phosphoribosyl)-5-[(5-phosphoribosylamino)methylideneamino)imidazole-4-carboxamide isomerase')]),
 ('y000012',
  '1-pyrroline-5-carboxylate dehydrogenase',
  [('P5CDm', '1 pyrroline 5 carboxylate dehydrogenase  mitochondrial')]),
 ('y000013',
  '2,3-diketo-5-methylthio-1-phosphopentane degradation reaction',
  [('DKMPPD2',
    '2 3 diketo 5 methylthio 1 phosphopentane degradation reaction')]),
 ('y000016',
  '2-aceto-2-hydroxybutanoate synthase',
  [('ACHBSm', '2 aceto 2 hydroxybutanoate synthase  mitochondrial')]),
 ('AATA',
  '2-aminoadipate transaminase',
  [('AATA', '2 aminoadipate transaminase')]),
 ('DPR',
  '2-dehydropantoate 2-reductase',
  [('DPR', '2-dehydropantoate 2-reductase')]),
 ('y000020',
  '2-deoxy-D-arabino-heptulosonate 7-phosphate synthetase',
  [('DDPAm',
    '2 deoxy D arabino heptulosonate 7 phosphate synthetase  mitochondria

In [98]:
pattern = re.compile('y[0-9]+')

In [99]:
ambiguous_mapping = []
duplicate_mapping = []
for mapping in suggested_rxn_mappings:
    if len(mapping[2]) != 1:
        ambiguous_mapping.append(mapping)
    else:
        yali_rxn = yali.reactions.get_by_id(mapping[0])
        imm904_rxn = iMM904.reactions.get_by_id(mapping[2][0][0])
        yali_rxn.annotation.setdefault('bigg.reaction', imm904_rxn.annotation['bigg.reaction'])
        yali_rxn.annotation['bigg.reaction'] = imm904_rxn.annotation['bigg.reaction']
        if re.findall(pattern, yali_rxn.id):
            yali_rxn.annotation["standard.metabolite"] = yali_rxn.id
        try: 
            yali_rxn.id = yali_rxn.annotation["bigg.reaction"]
            yali.repair()
        except ValueError:
            duplicate_mapping.append(yali_rxn)

In [100]:
pattern = re.compile('y[0-9]+')
unmapped_rxns = [rxn for rxn in yali.reactions if re.findall(pattern, rxn.id)]
len(unmapped_rxns)

1033

In [117]:
from cobra.io import save_yaml_model, write_sbml_model, save_json_model

In [116]:
# write_sbml_model(yali, "../iYali-test.xml")
# save_json_model(yali, "../iYali-test.json")

In [119]:
write_sbml_model(yali, "../ModelFiles/xml/iYali.xml")
save_yaml_model(yali, "../ModelFiles/yml/iYali.yml", sort=True)